In [0]:
# Table name
table_name = "housing"

# Load data from the table
housing_df = spark.read.table(table_name)

In [0]:
housing_df.display()

In [0]:
housing_df.count()

In [0]:
housing_df.columns

In [0]:
housing_df.dtypes

In [0]:
type(housing_df)

In [0]:
train_df, test_df = housing_df.randomSplit([0.99, 0.01], seed=42)

In [0]:
from databricks import automl

In [0]:
summary = automl.regress(train_df,target_col="median_house_value",timeout_minutes=5)

In [0]:
print(summary)

In [0]:
print(summary.best_trial.model_path)

Lets test the model

In [0]:
import mlflow

In [0]:
model_uri = f"runs:/{summary.best_trial.mlflow_run_id}/model"
predict = mlflow.pyfunc.spark_udf(spark, model_uri)
pred_df = test_df.withColumn("prediction", predict(*test_df.drop("median_house_value").columns))
display(pred_df)

In [0]:
from pyspark.ml.evaluation import RegressionEvaluator
regression_evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="median_house_value", metricName="r2")
rmse = regression_evaluator.evaluate(pred_df)
print(f"val_r2_score on test dataset: {rmse:.3f}")